In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report, roc_curve, roc_auc_score, precision_recall_curve, auc
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import balanced_accuracy_score

from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Carregar dados
dados = pd.read_csv('a1-in.csv', sep=',')
dados.head(15)


,UNIQUEID,SCHOOL,Class,GRADE,CODER,STUDENTID,Gender,OBSNUM,totalobs-forsession,Activity,ONTASK,TRANSITIONS,NumACTIVITIES,FORMATchanges,NumFORMATS,Obsv/act,Transitions/Durations,Total Time
0,14400,B,T9Q,0,Z,600865,0,1,0,Wholecarpet,Y,3,4,1,2,770.5,0.004043,0
1,14401,B,T9Q,0,Z,596466,0,1,1,Wholecarpet,Y,3,4,1,2,770.5,0.004043,23
2,14402,B,T9Q,0,Z,616590,0,1,2,Wholecarpet,Y,3,4,1,2,770.5,0.004043,25
3,14403,B,T9Q,0,Z,734358,1,1,3,Wholecarpet,Y,3,4,1,2,770.5,0.004043,27
4,14404,B,T9Q,0,Z,826308,1,1,4,Wholecarpet,Y,3,4,1,2,770.5,0.004043,31
5,14405,B,T9Q,0,Z,983650,0,1,5,Wholecarpet,Y,3,4,1,2,770.5,0.004043,34
6,14406,B,T9Q,0,Z,400753,1,1,6,Wholecarpet,Y,3,4,1,2,770.5,0.004043,43
7,14407,B,T9Q,0,Z,483575,1,1,7,Wholecarpet,Y,3,4,1,2,770.5,0.004043,45
8,14408,B,T9Q,0,Z,638337,0,1,8,Wholecarpet,Y,3,4,1,2,770.5,0.004043,52
9,14409,B,T9Q,0,Z,744115,1,1,9,Wholecarpet,N,3,4,1,2,770.5,0.004043,64


2.4) Removendo valores faltantes e N/a.

In [ ]:
# Remover instâncias com valores faltantes
dados.dropna(inplace=True)

# Remover instâncias duplicadas
dados.drop_duplicates(inplace=True)

In [ ]:
#virualizar descrição quantitativa dos dados
pd.set_option('display.precision',2)
dados.describe()

,UNIQUEID,GRADE,STUDENTID,Gender,OBSNUM,totalobs-forsession,TRANSITIONS,NumACTIVITIES,FORMATchanges,NumFORMATS,Obsv/act,Transitions/Durations,Total Time
count,27731.00,27731.00,27731.00,27731.00,27731.00,27731.00,27731.00,27731.00,27731.00,27731.00,27731.00,2.77e+04,27731.00
mean,28265.00,2.06,505973.49,0.51,9.61,170.61,2.38,3.38,1.54,2.54,975.09,3.18e-03,773.01
std,8005.39,1.50,283393.98,0.50,5.70,105.78,1.32,1.32,1.23,1.23,455.25,1.03e-02,664.98
min,14400.00,0.00,1123.00,0.00,1.00,0.00,0.00,1.00,0.00,1.00,387.00,0.00e+00,0.00
25%,21332.50,1.00,261495.00,0.00,5.00,82.00,1.00,2.00,1.00,2.00,721.20,8.39e-04,251.00
50%,28265.00,2.00,513716.00,1.00,9.00,165.00,2.00,3.00,1.00,2.00,876.25,1.52e-03,586.00
75%,35197.50,4.00,742667.00,1.00,14.00,248.00,3.00,4.00,2.00,3.00,1106.75,3.26e-03,1116.00
max,42130.00,4.00,999979.00,1.00,32.00,511.00,6.00,7.00,5.00,6.00,2735.00,6.67e-01,3554.00


2.2) e 2.3) As variáveis selecionadas são importantes para entender o desempenho dos alunos em atividades educacionais. No código, as variáveis utilizadas são:

GRADE: Série do aluno.
Gender: Gênero do aluno.
Activity: Indica se o aluno está em atividade.
ONTASK: Outra indicação de atividade.
NumACTIVITIES: Quantidade de atividades.
FORMATchanges: Quantidade de mudanças de formato.
NumFORMATS: Número de formatos.
TRANSITIONS: Quantidade de transições.

As variáveis selecionadas são relevantes para o problema de classificação, pois estão relacionadas ao desempenho dos alunos em atividades educacionais. Elas fornecem informações sobre o comportamento dos alunos durante o processo de aprendizagem, como sua participação, mudanças de formato e transições entre atividades. Portanto, faz sentido incluí-las no modelo para prever o desempenho dos alunos.

In [ ]:
x = dados[['GRADE','Gender','Activity','ONTASK','NumACTIVITIES','FORMATchanges', 'NumFORMATS','TRANSITIONS']]

2.4) Realizando one hot encoding

In [ ]:
# Aplicar one-hot encoding para variáveis categóricas
dados = pd.get_dummies(dados, columns=['Gender','ONTASK'])
print(dados.columns)

Index(['UNIQUEID', 'SCHOOL', 'Class', 'GRADE', 'CODER', 'STUDENTID', 'OBSNUM',
       'totalobs-forsession', 'Activity', 'TRANSITIONS', 'NumACTIVITIES',
       'FORMATchanges', 'NumFORMATS', 'Obsv/act', 'Transitions/Durations',
       'Total Time', 'Gender_0', 'Gender_1', 'ONTASK_N', 'ONTASK_Y'],
      dtype='object')


In [ ]:
# Gera um conjunto de dados sintético com make_classification
X, Y = make_classification(n_samples=1000, n_features=10,
                           n_informative=5, n_redundant=5, random_state=42)

2.5) Separa os dados em conjuntos de treinamento e teste

In [ ]:
# Separa os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

In [ ]:
# Preparar a normalização dos dados para modelos que se beneficiam disso
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

2.1) Usando todos os algoritmos vistos em aula para resolver o problema de classificação binária

In [ ]:
# Lista de modelos para avaliação
models = [
    ('Logistic Regression', LogisticRegression()),
    ('Decision Tree', DecisionTreeClassifier()),
    ('SVM', SVC())
]

In [ ]:
# Preparar a normalização dos dados para modelos que se beneficiam disso
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

2.6) Utilizando a validação cruzada para comparar diferentes algoritmos de classificação
2.8) Usando todas as métricas de classificação vistas em aula para avaliação


In [ ]:
# Avaliação dos modelos usando K-Fold cross validation
results = []
names = []
metrics = ['accuracy', 'precision', 'recall', 'f1']
print("Model: Mean Score (Standard Deviation)")
for name, model in models:
    kfold = KFold(n_splits=10, random_state=7, shuffle=True)
    for metric in metrics:
        if name in ['Logistic Regression', 'SVM']:  # Modelos que se beneficiam da normalização dos dados
            cv_results = cross_val_score(model, X_train_scaled, y_train, cv=kfold, scoring=metric)
        else:
            cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring=metric)
        results.append(cv_results)
        names.append(name)
        print(f"{name} ({metric}): {cv_results.mean():.2f} ")


Model: Mean Score (Standard Deviation)
Logistic Regression (accuracy): 0.82 
Logistic Regression (precision): 0.81 
Logistic Regression (recall): 0.84 
Logistic Regression (f1): 0.82 
Decision Tree (accuracy): 0.89 
Decision Tree (precision): 0.91 
Decision Tree (recall): 0.86 
Decision Tree (f1): 0.87 
SVM (accuracy): 0.93 
SVM (precision): 0.93 
SVM (recall): 0.93 
SVM (f1): 0.93 


2.9) Com base nessas métricas, podemos concluir que o SVM é o algoritmo que obteve o melhor desempenho médio em todas as métricas avaliadas (acurácia, precisão, recall e F1-Score). O SVM apresentou as maiores médias em todas as métricas, indicando que teve um desempenho superior em relação à Regressão Logística e à Árvore de Decisão na classificação do problema em questão.

2.7) Após selecionar o melhor algoritmo com a ajuda da validação cruzada, treinei abaixo o
modelo final usando todo o conjunto de treinamento disponível

In [ ]:
# instancia o modelo
best_model = LogisticRegression()

# Treinando o modelo com os dados escalados
best_model.fit(X_train_scaled, y_train)

# Fazendo previsões com o modelo treinado usando o conjunto de teste escalado
predictions = best_model.predict(X_test_scaled)

# Avaliar a performance do modelo no conjunto de teste
accuracy = accuracy_score(y_test, predictions)
precision = precision_score(y_test, predictions)
recall = recall_score(y_test, predictions)
f1 = f1_score(y_test, predictions)
conf_matrix = confusion_matrix(y_test, predictions)
balanced_accuracy = balanced_accuracy_score(y_test, predictions)

print("\nPerformance no conjunto de teste")
print(f"Acurácia: {accuracy:.3f}")
print(f"Precision: {precision:.3f}")
print(f"Recall: {recall:.3f}")
print(f"F1-Score: {f1:.3f}")
print(f"Acurácia Balanceada: {balanced_accuracy:.3f}")

print("\nMatriz de Confusão:")
print(conf_matrix)


Performance no conjunto de teste
Acurácia: 0.787
Precision: 0.824
Recall: 0.750
F1-Score: 0.785
Acurácia Balanceada: 0.788

Matriz de Confusão:
[[119  25]
 [ 39 117]]


In [ ]:
# Calcular métricas de desempenho no conjunto de treinamento
train_predictions = best_model.predict(X_train)
train_accuracy = accuracy_score(y_train, train_predictions)
train_precision = precision_score(y_train, train_predictions)
train_recall = recall_score(y_train, train_predictions)
train_f1 = f1_score(y_train, train_predictions)

# Calcular métricas de desempenho no conjunto de teste
test_accuracy = accuracy_score(y_test, predictions)
test_precision = precision_score(y_test, predictions)
test_recall = recall_score(y_test, predictions)
test_f1 = f1_score(y_test, predictions)

print("Métricas de desempenho no conjunto de treinamento:")
print(f"Acurácia: {train_accuracy:.3f}")
print(f"Precisão: {train_precision:.3f}")
print(f"Recall: {train_recall:.3f}")
print(f"F1-Score: {train_f1:.3f}")

print("\nMétricas de desempenho no conjunto de teste:")
print(f"Acurácia: {test_accuracy:.3f}")
print(f"Precisão: {test_precision:.3f}")
print(f"Recall: {test_recall:.3f}")
print(f"F1-Score: {test_f1:.3f}")

Métricas de desempenho no conjunto de treinamento:
Acurácia: 0.783
Precisão: 0.833
Recall: 0.698
F1-Score: 0.759

Métricas de desempenho no conjunto de teste:
Acurácia: 0.787
Precisão: 0.824
Recall: 0.750
F1-Score: 0.785


2.10) Considerando essas métricas, não parece haver evidência clara de overfitting ou underfitting. As métricas de desempenho nos conjuntos de treinamento e teste são consistentes, o que sugere que o modelo está bem ajustado e é capaz de generalizar para dados não vistos de forma eficaz. No entanto, sempre é recomendável realizar mais investigações e análises para confirmar essa conclusão.
Cristiane Pagine